<a href="https://colab.research.google.com/github/KhoomeiK/MindMapResearch/blob/master/HAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATA PREP STUFF

In [0]:
# download dataset and labels
! pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# downloaded1 = drive.CreateFile({'id': '1oZb283stxpZn8Dn8i8e2Vh6P8d6Voj4Y'}) 
# downloaded1.GetContentFile('dataset.zip')  

downloaded2 = drive.CreateFile({'id': '1-1nQU2lUwBnEyNot0EeVK72X92bdqVAu'}) 
downloaded2.GetContentFile('labels.pkl')

downloaded = drive.CreateFile({'id': '1-XYU2MCNbhS8ir_7ToC5DvrUMsQy9-9E'}) 
downloaded.GetContentFile('embeddings.zip')

# ! unzip dataset.zip
# ! rm -rf cse198f_shiv/data
# ! rm -rf cse198f_shiv/diagnostics
# ! rm -rf cse198f_shiv/models
# ! rm cse198f_shiv/vectors.py
# ! ls cse198f_shiv

In [1]:
# read dataset into memory
from os import listdir
from os.path import isfile, join
import pandas as pd

mypath = 'cse198f_shiv'
csvs = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(len(csvs))

data = []
names = []
for csv in csvs:
    if csv[-4:] == '.csv':
        try:
            data.append(pd.read_csv(join(mypath, csv), encoding='CP1252'))
            names.append(csv[:-4])
        except:
            try:
                data.append(pd.read_csv(join(mypath, csv), encoding='UTF8'))
                names.append(csv[:-4])
            except:
                continue
print(len(data))

# pd.reset_option('all')
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

2052
1980


In [0]:
# # download embedding tools
# ! ls *
# ! mkdir fastText
# ! curl https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip > fastText/crawl-300d-2M.vec.zip
# ! unzip fastText/crawl-300d-2M.vec.zip -d fastText/
# ! mkdir encoder
# ! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
# ! curl https://raw.githubusercontent.com/facebookresearch/InferSent/master/models.py > models.py
# ! mkdir embeddings

In [0]:
# generate embeddings of dataset
import torch, os
import numpy as np
import tensorflow_hub as hub

from absl import logging
from models import InferSent

import pickle
import time

MODEL_PATH = 'encoder/infersent2.pkl'
W2V_PATH = 'fastText/crawl-300d-2M.vec'

def load_infersent_model(model_path=MODEL_PATH, word_embeddings_path=W2V_PATH):
	params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048, 'pool_type': 'max', 'dpout_model': 0.0, 'version': 2}
	infersent = InferSent(params_model).cuda()
	infersent.load_state_dict(torch.load(model_path))
	infersent.set_w2v_path(word_embeddings_path)
	infersent.build_vocab_k_words(K=100000)
	return infersent

def get_infersent_vectors(sentences, model):
	return model.encode(sentences, tokenize=False, verbose=False)

def get_user_data_embeddings(comments, model):
	# model = load_infersent_model()
	embedding = get_infersent_vectors(comments, model)
	return embedding

dataEmbeddings = []
model = load_infersent_model()

start = time.time()
for i, name in enumerate(names):
	comments = list(data[i]['text']) if 'text' in data[i] else []
	if len(comments) >= 1:
		print(i, name)
		try:
			embeddings = get_user_data_embeddings(data[i]['text'], model)
			print(len(embeddings), 'comments')
			dataEmbeddings.append(embeddings)
			with open('embeddings/%s.pkl' % name, 'wb') as pkl:
				pickle.dump(embeddings, pkl)
		except:
			print('ERROR')

print(time.time() - start)

## MODEL RUN

In [6]:
# load labels and user embeddings and create Users data obj
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from csv import reader
import pickle
from os import listdir

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Users(Dataset):
    def __init__(self, embedPath, labelPath): # TODO: data reading
        self.users, self.labels = [], []
        users = {}
        labels = pickle.load(open(labelPath, 'rb')) # { username: [ depressionPercent, vaderScore ] }
        for user in listdir(embedPath): # must download and unzip embeddings.zip first
            users[user[:-4]] = pickle.load(open('%s/%s' % (embedPath, user), 'rb'))
        
        intersection = list(set(users.keys()).intersection(set(labels.keys())))
        for i in intersection:
            self.users.append(torch.tensor(users[i]))
            self.labels.append(torch.tensor(labels[i]))

        print(len(self), self[0])

    def __getitem__(self, i):
        return self.users[i], self.labels[i][0] # remove [0]
    
    def __len__(self):
        assert len(self.users) == len(self.labels)
        return len(self.users)

# ! unzip embeddings.zip
data = DataLoader(Users('embeddings', 'labels.pkl'))

1765 (tensor([[ 0.0075, -0.0307,  0.1035,  ..., -0.0002, -0.0056,  0.0229],
        [ 0.0075, -0.0278,  0.0028,  ...,  0.0186,  0.0176, -0.0132],
        [ 0.0075, -0.0413, -0.0464,  ..., -0.0070, -0.0356, -0.0231],
        ...,
        [ 0.0075, -0.0322, -0.0456,  ..., -0.0044, -0.0541, -0.0285],
        [ 0.0075, -0.1444, -0.0628,  ..., -0.1045, -0.0613, -0.0468],
        [ 0.0075, -0.1444, -0.0628,  ..., -0.1045, -0.0613, -0.0468]]), tensor(0.0040))


In [78]:
class HAN(nn.Module):
    def __init__(
        self,
        batch_size=8,
        embedding_dimension=4096, # from glove to infersent?
        hidden_size=150, 
        n_layers=1, # multiple?
    ):
        super(HAN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        '''
        GRUforward(sentences)
        GRUbackward(sentences)
        concat
        attention(each sentence state) # basically a post vector
        GRUforward(post vectors)
        # backward?
        attention(each post state)
        linear output
        '''

        # self.gru1 = nn.GRU( # pass through sent vecs
        #     embedding_dimension,
        #     hidden_size,
        #     num_layers=n_layers,
        #     bidirectional=True
        # )

        # # https://www.cc.gatech.edu/~dyang888/docs/naacl16.pdf
        # self.mlp1 = nn.Linear(hidden_size * 2, hidden_size) # attention mlp
        # self.contextW1 = nn.Parameter(torch.Tensor(hidden_size * 2, 1)) # attention weights

        self.gru2 = nn.GRU( # pass through post vecs
            embedding_dimension,
            hidden_size,
            num_layers=n_layers,
            bidirectional=True
        )
        
        self.mlp2 = nn.Linear(hidden_size * 2, hidden_size * 2) # attention mlp
        self.contextW2 = nn.Parameter(torch.Tensor(hidden_size * 2, 1)) # attention weights
    
    def init_hidden(self):
        pass

    def forward(self, user):
        # postVecs = torch.tensor([])
        # for post in user:
        #     sentAnnot = self.gru1(post) # tensor of sentence annotations of post
        #     sentU = F.Tanh(self.mlp1(sentAnnot)) # hidden reps of sentAnnots
        #     sentA = F.Softmax(sentU * self.contextW1) # att weights of sents
        #     postVec = torch.sum(sentA * sentAnnot) # total val of post; 1 sum?
        #     torch.cat((postVecs, postVec))
        # print('user', user)

        postAnnot, postSplitAnnot = self.gru2(user)
        print('postAnnot & postSplitAnnot', postAnnot.shape, postSplitAnnot.shape)
        postU = F.tanh(self.mlp2(postAnnot)) # hidden reps of postAnnots
        print('postU & context', postU.shape, self.contextW2.shape)
        # for u in postU[0]:
        #     postA = F.softmax(u * self.contextW2) # att weights of posts
        # postA = F.softmax(postU * self.contextW2) # att weights of posts
        temp = [F.softmax(u * self.contextW2) for u in postU[0]]
        # print(postA)
        postA = torch.stack(temp)
        print(postA.shape)
        postB = torch.stack(temp)
        print(postB.shape)
        userVec = torch.sum(postA * postAnnot) # total val of user; 1 sum?

        output = F.softmax(userVec) # classify
        
        return output

adc.json  embeddings  embeddings.zip  labels.pkl  sample_data


In [73]:
model = HAN().to(device)
# data = data.to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()
losses = []
model.train()

for epoch in range(4):
    total = 0
    for X, Y in data:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, Y)
        loss.backward()
        optimizer.step()

        total += 1
        losses.append(loss.item())
        print(loss)
        # data.set_description(f'Loss: {loss.item():.3f}')
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)

    # tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

postAnnot & postSplitAnnot torch.Size([1, 100, 300]) torch.Size([2, 100, 150])
postU & context torch.Size([1, 100, 300]) torch.Size([300, 1])
torch.Size([100, 300, 300])
torch.Size([100, 300, 300])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: ignored